**Livrarias**

In [ ]:
import pandas as pd
#!pip install pandas-profiling
#import pandas_profiling
import plotly.graph_objects as go
from plotly.subplots import make_subplots

**Carregamento de dados**

In [ ]:
ruta_archivo = '/content/db_salud_ini.xlsx'
datos = pd.read_excel(ruta_archivo)

**Pré-tratamento de dados**

In [ ]:
dimension_inicial = datos.shape
filas_con_na = datos[datos.isna().any(axis=1)]
filas_con_asterisco = datos[datos.apply(lambda x: any('*' in str(val) for val in x), axis=1)]

# Imprimir las filas con valores NaN (NA)
print("As linhas com NA são:")
for indice_fila in filas_con_na.index:
    print(f"- {indice_fila}")

# Imprimir las filas con asteriscos (*)
print("As linhas com asteriscos (*) são:")
for indice_fila in filas_con_asterisco.index:
    print(f"- {indice_fila}")    

As linhas com NA são:
As linhas com asteriscos (*) são:


In [ ]:
print(datos.dtypes)
variables_a_eliminar = ['patient_#', 'date_d_test', 'pre-infection_category', 'predicted_annual_mean_category', 'predicted_level_category', 'predicted_annual_mean','predicted_monthly_mean', 'predicted_level']
datos_corr = datos.drop(variables_a_eliminar, axis=1)

patient_#                                     int64
age_group                                     int64
gender                                        int64
religion                                    float64
max_dg_sev_during_hospitalization             int64
max_dg_sev_categories                         int64
BMI                                           int64
date_d_test                          datetime64[ns]
month_d_examination                           int64
month_hosp_corona_dep                         int64
death_during_hospitalization                  int64
COPD                                          int64
CHF                                           int64
CIHD                                          int64
HTN                                           int64
CRF                                           int64
DM                                            int64
Hospitalization_length_days                   int64
pre-infection                               float64
pre-infectio

In [ ]:
print(datos_corr.dtypes)

age_group                              int64
gender                                 int64
religion                             float64
max_dg_sev_during_hospitalization      int64
max_dg_sev_categories                  int64
BMI                                    int64
month_d_examination                    int64
month_hosp_corona_dep                  int64
death_during_hospitalization           int64
COPD                                   int64
CHF                                    int64
CIHD                                   int64
HTN                                    int64
CRF                                    int64
DM                                     int64
Hospitalization_length_days            int64
pre-infection                        float64
dtype: object


In [ ]:
# Combinar índices de filas a eliminar
filas_a_eliminar = filas_con_na.index.union(filas_con_asterisco.index)

# Eliminar las filas del dataset
datos = datos.drop(filas_a_eliminar)

dimension_actualizada = datos.shape

# Calcular el porcentaje de alteración en la dimensión
porcentaje_alterado = (dimension_inicial[0] - dimension_actualizada[0]) / dimension_inicial[0] * 100

# Crear tabla de resumen
tabla_resumen = pd.DataFrame({'# Linhas': [dimension_inicial[0], dimension_actualizada[0]],
                              '# Colunas': [dimension_inicial[1], dimension_actualizada[1]],
                              '% Alterado': ['0%', f'{porcentaje_alterado:.2f}%']},
                             index=['Antes', 'Depois'])

# Imprimir tabla de resumen
print("Resumo das dimensões:")
print(tabla_resumen)

Resumo das dimensões:
        # Linhas  # Colunas % Alterado
Antes        246         25         0%
Depois       246         25      0.00%


**EDA - Modelo 1**

In [ ]:
# Generar el informe EDA
perfil_datos = pandas_profiling.ProfileReport(datos)

# Exportar el informe como un archivo HTML

ruta_informe_html = '/content/informe_eda.html'
perfil_datos.to_file(output_file=ruta_informe_html)

**EDA - Modelo 2**

In [ ]:
# Mapear las nuevas etiquetas a los valores existentes en la columna 'age_group'
age_labels = {1: '<50', 2: '50-64', 3: '>64'}
temp_df = datos[['age_group', 'gender']].copy()
temp_df['age_group'] = temp_df['age_group'].map(age_labels)

grouped_counts = temp_df.groupby(['age_group', 'gender']).size().reset_index(name='Count')

fig = go.Figure(data=[
    go.Bar(name='Masculino', x=grouped_counts[grouped_counts['gender'] == 0]['age_group'], y=grouped_counts[grouped_counts['gender'] == 0]['Count']),
    go.Bar(name='Feminino', x=grouped_counts[grouped_counts['gender'] == 1]['age_group'], y=grouped_counts[grouped_counts['gender'] == 1]['Count'])
])

fig.update_layout(
    title='Distribuição dos pacientes por faixa etária e gênero',
    xaxis=dict(title='Faixa etária'),
    yaxis=dict(title='Quantidade'),
    barmode='group',
    legend=dict(orientation='h', yanchor='bottom', y=1.02, xanchor='right', x=1)
)

fig.show()

In [ ]:
# Mapear las nuevas etiquetas a los valores existentes en la columna 'max_dg_sev_during_hospitalization'
severity_labels = {0: 'Mild', 1: 'Moderate', 2: 'Severe', 3: 'Critical'}
temp_df = datos[['max_dg_sev_during_hospitalization']].copy()
severity_colors = {'Mild': '#00bfff', 'Moderate': '#ffa500', 'Severe': '#ff0000', 'Critical': '#633974'}
temp_df['max_dg_sev_during_hospitalization'] = temp_df['max_dg_sev_during_hospitalization'].map(severity_labels)

severity_counts = temp_df['max_dg_sev_during_hospitalization'].value_counts().reset_index().rename(columns={'index': 'Severity', 'max_dg_sev_during_hospitalization': 'Count'})

fig = go.Figure(data=[
    go.Bar(x=severity_counts['Severity'], y=severity_counts['Count'], marker_color=[severity_colors[severity] for severity in severity_counts['Severity']])
])

fig.update_layout(
    title='Gravidade máxima do diagnóstico durante a hospitalização',
    xaxis=dict(title='Gravidade'),
    yaxis=dict(title='Quantidade'),
)

fig.show()

In [ ]:
import pandas as pd
import plotly.graph_objects as go

# Mapear las nuevas etiquetas a los valores existentes en la columna 'pre-infection_category'
pre_infection_labels = {1: '<=19.99', 2: '20-29.99', 3: '30-39.99', 4: '40+'}
temp_df = datos[['pre-infection_category', 'gender']].copy()
temp_df['pre-infection_category'] = temp_df['pre-infection_category'].map(pre_infection_labels)

# Agrupar los datos por 'pre-infection_category' y 'gender' y calcular los conteos
grouped_counts = temp_df.groupby(['pre-infection_category', 'gender']).size().reset_index(name='Count')

# Mapear las etiquetas de género a valores descriptivos
gender_labels = {0: 'Masculino', 1: 'Femenino'}
grouped_counts['gender'] = grouped_counts['gender'].map(gender_labels)

# Crear el gráfico de barras
fig = go.Figure(data=[
    go.Bar(name='Masculino', x=grouped_counts[grouped_counts['gender'] == 'Masculino']['pre-infection_category'], y=grouped_counts[grouped_counts['gender'] == 'Masculino']['Count']),
    go.Bar(name='Femenino', x=grouped_counts[grouped_counts['gender'] == 'Femenino']['pre-infection_category'], y=grouped_counts[grouped_counts['gender'] == 'Femenino']['Count'])
])

# Actualizar el diseño del gráfico
fig.update_layout(
    title='Distribuição da categoria pré-infecção por gênero',
    xaxis=dict(title='Categoria pré-infecção'),
    yaxis=dict(title='Quantidade'),
    barmode='group',
    legend=dict(orientation='h', yanchor='bottom', y=1.02, xanchor='right', x=1)
)

# Mostrar el gráfico
fig.show()

In [ ]:
import plotly.graph_objects as go

# Mapear las etiquetas de la categoría de pre-infección
pre_infection_labels = {1: '<=19.99', 2: '20-29.99', 3: '30-39.99', 4: '40+'}
datos['pre-infection_category'] = datos['pre-infection_category'].map(pre_infection_labels)

# Mapear las etiquetas de religión
religion_labels = {1: 'Judeu', 2: 'Árabe muçulmano', 3: 'Árabe Cristão', 4: 'Druzo', 5: 'Cristão', 6: 'Outra'}
datos['religion'] = datos['religion'].map(religion_labels)

# Agrupar los datos por 'pre-infection_category' y 'religion' y calcular los conteos
grouped_counts = datos.groupby(['pre-infection_category', 'religion']).size().reset_index(name='Count')

# Crear el gráfico de barras
fig = go.Figure()

# Iterar sobre las categorías de religión y agregar barras al gráfico
for religion in religion_labels.values():
    religion_data = grouped_counts[grouped_counts['religion'] == religion]
    fig.add_trace(go.Bar(name=religion, x=religion_data['pre-infection_category'], y=religion_data['Count']))

# Actualizar el diseño del gráfico
fig.update_layout(
    title='Distribuição da categoria pré-infecção por religião',
    xaxis=dict(title='Categoria pré-infecção'),
    yaxis=dict(title='Quantidade'),
    barmode='group',
    legend=dict(orientation='h', yanchor='bottom', y=1.02, xanchor='right', x=1)
)

# Mostrar el gráfico
fig.show()

In [ ]:
import pandas as pd
import plotly.graph_objects as go

# Crear una copia del DataFrame original
temp_df = datos.copy()

# Mapear las etiquetas de la categoría de pre-infección
pre_infection_labels = {1: '<=19.99', 2: '20-29.99', 3: '30-39.99', 4: '40+'}
temp_df['pre-infection_category'] = temp_df['pre-infection_category'].map(pre_infection_labels)

# Mapear las etiquetas de 'death_during_hospitalization'
death_labels = {0: 'Não', 1: 'Sim'}
temp_df['death_during_hospitalization'] = temp_df['death_during_hospitalization'].map(death_labels)

# Agrupar los datos por 'pre-infection_category' y 'death_during_hospitalization' y calcular los conteos
grouped_counts = temp_df.groupby(['pre-infection_category', 'death_during_hospitalization']).size().reset_index(name='Count')

# Crear el gráfico de barras
fig = go.Figure()

# Iterar sobre las categorías de 'death_during_hospitalization' y agregar barras al gráfico
for death in death_labels.values():
    death_data = grouped_counts[grouped_counts['death_during_hospitalization'] == death]
    fig.add_trace(go.Bar(name=death, x=death_data['pre-infection_category'], y=death_data['Count']))

# Actualizar el diseño del gráfico
fig.update_layout(
    title='Relação entre a categoria pré-infecção e a morte durante a hospitalização',
    xaxis=dict(title='Categoria pré-infecção'),
    yaxis=dict(title='Quantidade'),
    barmode='group',
    legend=dict(orientation='h', yanchor='bottom', y=1.02, xanchor='right', x=1)
)

# Mostrar el gráfico
fig.show()

In [ ]:
datos_corr.head()

,age_group,gender,religion,max_dg_sev_during_hospitalization,max_dg_sev_categories,BMI,date_d_test,month_d_examination,month_hosp_corona_dep,death_during_hospitalization,COPD,CHF,CIHD,HTN,CRF,DM,Hospitalization_length_days,pre-infection
0,3,0,1.0,1,0,0,2019-09-22,9,10,0,1,0,0,1,0,1,3,17.3
1,3,1,6.0,0,0,1,2018-06-25,6,4,0,1,0,1,1,0,1,13,42.0
2,3,1,1.0,0,0,0,2020-04-01,4,10,0,1,0,0,1,0,0,6,34.2
3,2,0,2.0,1,0,0,2019-10-01,10,11,0,1,0,0,1,0,1,8,17.2
4,3,0,1.0,0,0,0,2019-06-18,6,7,0,1,0,0,1,0,0,10,18.5


In [ ]:
correlation_matrix = datos_corr.corr()

# Crear el mapa de calor
fig = go.Figure(data=go.Heatmap(
    z=correlation_matrix.values,
    x=correlation_matrix.columns,
    y=correlation_matrix.columns,
    colorscale='RdBu'))

# Actualizar el diseño del mapa de calor
fig.update_layout(
    title='Matriz de correlação',
    xaxis=dict(title='Variáveis'),
    yaxis=dict(title='Variáveis'))

# Mostrar el mapa de calor
fig.show()

<ipython-input-224-9d9e063f4165>:1: FutureWarning:

The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.



In [ ]:
import numpy as np
import plotly.graph_objects as go

# Calcular la matriz de correlación
correlation_matrix = datos_corr.corr()

# Obtener la mitad superior de la matriz de correlación
mask = np.triu(np.ones(correlation_matrix.shape, dtype=bool))
upper_triangle = correlation_matrix.mask(mask).round(2)

# Crear el gráfico de correlación con Plotly
fig = go.Figure(data=go.Heatmap(z=upper_triangle.values,
                               x=upper_triangle.columns,
                               y=upper_triangle.index,
                               colorscale='RdBu_r',
                               text=upper_triangle.values,
                               hovertemplate='Correlación: %{text}<extra></extra>'))

# Actualizar el diseño del gráfico
fig.update_layout(
    title='Gráfico de Correlación',
    xaxis_title='Variables',
    yaxis_title='Variables',
)

# Mostrar el gráfico
fig.show()

In [ ]:
correlation_matrix = datos_corr.corr()

# Convertir la matriz de correlación en un DataFrame
correlation_df = pd.DataFrame(correlation_matrix)

# Obtener las combinaciones de correlaciones y sus índices
correlation_combinations = correlation_df.unstack().sort_values(ascending=False)

# Filtrar las combinaciones de correlaciones que no son 1 y cumplen el criterio A con B o B con A
unique_combinations = correlation_combinations[~correlation_combinations.index.duplicated()]
filtered_combinations = unique_combinations[(unique_combinations != 1) & (unique_combinations.index.get_level_values(0) != unique_combinations.index.get_level_values(1).str[::-1])]

# Crear una lista de tuplas con las combinaciones de correlaciones y sus índices
table_data = [(str(index), "{:.4f}".format(value)) for index, value in filtered_combinations.items()]

# Crear una lista de encabezados de columna
headers = ['Combinación de correlaciones', 'Valor']

# Mostrar la tabla con formato colorido
table = tabulate(table_data, headers=headers, tablefmt='fancy_grid')

# Imprimir la tabla con colores
colored_table = colored(table, color='cyan')
print(colored_table)

╒═══════════════════════════════════════════════════════════════════════╤═════════╕
│ Combinación de correlaciones                                          │   Valor │
╞═══════════════════════════════════════════════════════════════════════╪═════════╡
│ ('HTN', 'COPD')                                                       │  0.9838 │
├───────────────────────────────────────────────────────────────────────┼─────────┤
│ ('COPD', 'HTN')                                                       │  0.9838 │
├───────────────────────────────────────────────────────────────────────┼─────────┤
│ ('max_dg_sev_during_hospitalization', 'max_dg_sev_categories')        │  0.8585 │
├───────────────────────────────────────────────────────────────────────┼─────────┤
│ ('max_dg_sev_categories', 'max_dg_sev_during_hospitalization')        │  0.8585 │
├───────────────────────────────────────────────────────────────────────┼─────────┤
│ ('HTN', 'age_group')                                                  │  0

In [ ]:
import statsmodels.api as sm

df = pd.DataFrame(datos_corr)

# Separar las variables independientes (X) y la variable dependiente (y)
X = df.drop('pre-infection', axis=1)
y = df['pre-infection']

# Añadir una columna de unos para el término constante
X = sm.add_constant(X)

# Ajustar el modelo de regresión múltiple
model = sm.OLS(y, X).fit()

# Imprimir los resultados de la regresión
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:          pre-infection   R-squared:                       0.378
Model:                            OLS   Adj. R-squared:                  0.335
Method:                 Least Squares   F-statistic:                     8.711
Date:                Sat, 20 May 2023   Prob (F-statistic):           1.83e-16
Time:                        23:12:30   Log-Likelihood:                -1027.5
No. Observations:                 246   AIC:                             2089.
Df Residuals:                     229   BIC:                             2149.
Df Model:                          16                                         
Covariance Type:            nonrobust                                         
                                        coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------
const 

In [ ]:
df = pd.DataFrame(datos_corr)

# Separar las variables independientes (X) y la variable dependiente (y)
X = df.drop(['max_dg_sev_during_hospitalization', 'Hospitalization_length_days', 'pre-infection'], axis=1)
y = df['pre-infection']

# Ajustar el modelo de regresión múltiple
model = sm.OLS(y, X).fit()

# Imprimir los resultados de la regresión
print(model.summary())

                                 OLS Regression Results                                
Dep. Variable:          pre-infection   R-squared (uncentered):                   0.653
Model:                            OLS   Adj. R-squared (uncentered):              0.632
Method:                 Least Squares   F-statistic:                              31.16
Date:                Sat, 20 May 2023   Prob (F-statistic):                    1.64e-45
Time:                        23:16:32   Log-Likelihood:                         -1078.4
No. Observations:                 246   AIC:                                      2185.
Df Residuals:                     232   BIC:                                      2234.
Df Model:                          14                                                  
Covariance Type:            nonrobust                                                  
                                   coef    std err          t      P>|t|      [0.025      0.975]
-----------------------

In [ ]:
coefficients = model.params

# Imprimir los coeficientes
print(coefficients)

age_group                        4.263596
gender                          -1.905057
religion                         2.523301
max_dg_sev_categories          -12.831371
BMI                             -3.597440
month_d_examination              0.812395
month_hosp_corona_dep            1.517731
death_during_hospitalization    -2.318228
COPD                             0.893771
CHF                             -2.680702
CIHD                            -1.234789
HTN                             -3.310613
CRF                             -2.832492
DM                               3.533602
dtype: float64


In [ ]:
import statsmodels.api as sm
from statsmodels.robust import norm_est

df = pd.DataFrame(datos_corr)

# Separar las variables independientes (X) y la variable dependiente (y)
X = df.drop('pre-infection', axis=1)
y = df['pre-infection']

# Añadir una columna de unos para el término constante
X = sm.add_constant(X)

# Ajustar el modelo de regresión múltiple con robustez
robust_model = sm.RLM(y, X, M=norm_est.HuberT())
robust_results = robust_model.fit()

# Ajustar el modelo de regresión múltiple con generalización
gen_model = sm.GLM(y, X, family=sm.families.Gaussian())
gen_results = gen_model.fit()

# Imprimir los resultados de los modelos
print("Resultados del modelo de regresión múltiple con robustez:")
print(robust_results.summary())

print("Resultados del modelo de regresión múltiple con generalización:")
print(gen_results.summary())

ImportError: ignored

In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [ ]:
# Dividir los datos en variables predictoras (X) y variable respuesta (y)
X = datos.drop(['max_dg_sev_during_hospitalization', 'date_d_test'], axis=1)
y = datos['max_dg_sev_during_hospitalization']

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Escalar los datos
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Crear un modelo de SVM
model = SVC(C=1.0, kernel='rbf', gamma='scale')

# Entrenar el modelo
model.fit(X_train, y_train)

# Realizar predicciones en el conjunto de prueba
y_pred = model.predict(X_test)

# Calcular la precisión del modelo
accuracy = accuracy_score(y_test, y_pred)
print("Precisión del modelo:", accuracy)

Precisión del modelo: 0.8


In [ ]:
# Eliminar la columna 'date_d_test'
datos = datos.drop(['date_d_test'], axis=1)

# Dividir los datos en variables predictoras (X) y variable respuesta (y)
X = datos.drop(['max_dg_sev_during_hospitalization'], axis=1)
y = datos['max_dg_sev_during_hospitalization']

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Crear un modelo de Random Forest
model = RandomForestClassifier(n_estimators=100, random_state=42)

# Entrenar el modelo
model.fit(X_train, y_train)

# Realizar predicciones en el conjunto de prueba
y_pred = model.predict(X_test)

# Calcular la precisión del modelo
accuracy = accuracy_score(y_test, y_pred)
print("Precisión del modelo:", accuracy)

Precisión del modelo: 0.82


In [ ]:
from sklearn.ensemble import VotingClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [ ]:
# Dividir los datos en variables predictoras (X) y variable respuesta (y)
X = datos.drop(['max_dg_sev_during_hospitalization'], axis=1)
y = datos['max_dg_sev_during_hospitalization']

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Crear el modelo de SVM
model_svm = SVC(C=1.0, kernel='rbf', gamma='scale')
model_svm.fit(X_train, y_train)

# Crear el modelo de Random Forest
model_rf = RandomForestClassifier(n_estimators=100, random_state=42)
model_rf.fit(X_train, y_train)

# Crear el ensemble de modelos
ensemble = VotingClassifier(estimators=[('svm', model_svm), ('random_forest', model_rf)], voting='hard')

# Entrenar el ensemble
ensemble.fit(X_train, y_train)

# Realizar predicciones en el conjunto de prueba
y_pred_ensemble = ensemble.predict(X_test)

# Calcular la precisión del ensemble
accuracy_ensemble = accuracy_score(y_test, y_pred_ensemble)
print("Precisión del ensemble:", accuracy_ensemble)

Precisión del ensemble: 0.52


In [ ]:
from sklearn.linear_model import LogisticRegression

# Crear el modelo de regresión logística con solucionador 'liblinear' y más iteraciones
model_logistic = LogisticRegression(solver='liblinear', max_iter=1000)

# Entrenar el modelo de regresión logística
model_logistic.fit(X_train, y_train)

# Realizar predicciones con el modelo de regresión logística
y_pred_logistic = model_logistic.predict(X_test)

# Calcular la precisión del modelo de regresión logística
accuracy_logistic = accuracy_score(y_test, y_pred_logistic)
print("Precisión de regresión logística:", accuracy_logistic)

# Crear el ensemble de modelos
ensemble = VotingClassifier(estimators=[('svm', model_svm), ('random_forest', model_rf), ('logistic', model_logistic)], voting='hard')

# Entrenar el ensemble
ensemble.fit(X_train, y_train)

# Realizar predicciones con el ensemble
y_pred_ensemble = ensemble.predict(X_test)

# Calcular la precisión del ensemble
accuracy_ensemble = accuracy_score(y_test, y_pred_ensemble)
print("Precisión del ensemble:", accuracy_ensemble)

Precisión de regresión logística: 0.8
Precisión del ensemble: 0.78
